In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins

# Load your fraud detection dataset
# For example, assuming 'df' is your DataFrame
df1 = pd.read_csv('/content/TransactionDataset1.csv')
df2 = pd.read_csv('/content/use this.csv')

# Concatenate the two DataFrames
df = pd.concat([df1, df2], axis=1)

# User input for the specific user
user_name = input("Enter the username: ")

# Filter the DataFrame for the specified user
user_df = df[df['name'] == user_name]

# Print information for the user
print(f"User Information for {user_name}:")
print(f"Transaction Count in the Last 7 Days: {user_df['transaction_count_7_days'].iloc[0]}")
print(f"Average Expenditure: {user_df['average_expenditure'].iloc[0]}")
print(f"Comparison with Average Expenditure: {user_df['comparison_with_avg_expenditure'].iloc[0]}")
print(f"Transaction Amount: {user_df['transaction_amount'].iloc[0]}")
print(f"Addresses:\n{user_df['addresses'].iloc[0]}")
if not user_df.empty:
    # Print start city and end city
    print(f"Start City: {user_df['start city'].iloc[0]}")
    print(f"End City: {user_df['end city'].iloc[0]}")
else:
    print(f"No information found for user: {user_name}")

# Check if 'Money Sources' column is present in the DataFrame
if 'Money Sources' in user_df.columns:
    # Print Money Sources Distribution
    print(f"Money Sources Distribution:\n{user_df['Money Sources'].value_counts()}")
else:
    print("Column 'Money Sources' not found in the DataFrame.")

# Check if it's a fraud case
if 'fraud_indicator' in user_df.columns and user_df['fraud_indicator'].iloc[0] == 1:
    print("Fraud Case: Yes")
else:
    print("Fraud Case: No")

# Continue with visualizations if 'Money Sources' is present
if 'Money Sources' in user_df.columns:
    # Visualizations for average expenditure and comparison with average expenditure side by side
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Average Expenditure
    sns.barplot(x='fraud_indicator', y='average_expenditure', data=user_df, ax=axes[0], palette='Blues')
    axes[0].set_title(f'Average Expenditure by Fraud Status for {user_name}')
    axes[0].set_xlabel('Fraud Indicator')
    axes[0].set_ylabel('Average Expenditure')

    # Comparison with Average Expenditure
    sns.barplot(x='fraud_indicator', y='comparison_with_avg_expenditure', data=user_df, ax=axes[1], palette='Reds')
    axes[1].set_title(f'Comparison with Average Expenditure by Fraud Status for {user_name}')
    axes[1].set_xlabel('Fraud Indicator')
    axes[1].set_ylabel('Comparison with Average Expenditure')

    plt.show()

    # Scatter plot for transaction amount vs average expenditure
    plt.figure(figsize=(12, 6))
    sns.scatterplot(x='transaction_amount', y='average_expenditure', hue='fraud_indicator', data=user_df)
    plt.title(f'Transaction Amount vs Average Expenditure for {user_name}')
    plt.xlabel('Transaction Amount')
    plt.ylabel('Average Expenditure')
    plt.show()

    # Plotting the bar graph
    plt.figure(figsize=(3, 7))
    sns.barplot(x='transaction_time', y='transaction_amount', data=user_df, palette='viridis')
    plt.title(f'Transaction Amount by Hour of the Day for {user_name}')
    plt.xlabel('Hour of the Day (24-hour format)')
    plt.ylabel('Transaction Amount')
    plt.show()

# Load the additional dataset for location data
# Specify the file path to the location data file (CSV format)
file_path_location = '/content/set for location.csv'  # Change this to your actual file path

# Load the location data from the provided file path
try:
    location_df = pd.read_csv(file_path_location)
except FileNotFoundError:
    print(f"Error: File not found at the specified path: {file_path_location}")
    exit()

# Function to get coordinates based on username
def get_coordinates(username):
    try:
        user_data = location_df[location_df['name'] == username].iloc[0]
        start_coordinates = (user_data['start lat'], user_data['start lng'])
        end_coordinates = (user_data['end lat'], user_data['end lng'])
        return start_coordinates, end_coordinates
    except IndexError:
        return None

# Get coordinates for start and end based on the username
coordinates = get_coordinates(user_name)

if coordinates:
    start_coordinates, end_coordinates = coordinates

    # Create a map centered around the start coordinates
    mymap = folium.Map(location=start_coordinates, zoom_start=5)

    # Add markers for start and end coordinates
    folium.Marker(location=start_coordinates, popup='Start', icon=folium.Icon(color='green')).add_to(mymap)
    folium.Marker(location=end_coordinates, popup='End', icon=folium.Icon(color='red')).add_to(mymap)

    # Add a line between the two coordinates
    folium.PolyLine([start_coordinates, end_coordinates], color='blue', weight=2.5, opacity=1).add_to(mymap)

    # Display the map
    display(mymap)
else:
    print(f"No data found for the username: {user_name}")
